#### TASK 1 

You’re given raw data that contains many flaws. 
Please find 5 different flaws in the data and fix them. Try to find the most important flaws that require to be fixed in order to enable forecasting (task 2)

#### TASK 2

Please describe (no code is needed) what will be the process required for forecasting the future number of active patients per country.

Hence we have data saved in convenient way (one file for each country) for this task, I would like to try following approaches:
- classical time series forecasting algorithms of family (S)AR(I)MA(X), using only target feature as an input and forecasting future cases with autoregressive modeling. From my experience these models work not so good on complex data (e.g. stocks) but may work good with this data. Of course, deep analysis should be done including trend/sesonality/stationarity checks. Based on this analysis we'll be able to understand what would be an optimal way of representing time series in autoregressive models.
- jumping from time series task to a task of supervised learning - this is my favorite apporoach of forecasting time series and from my experience this works great on complex data. Let's stick to this method in following chapters.

So the task is to represent data for supervised learning algorithm. We have to decide horison of forecasting - 
let's assume 1 for simplicity (means we'll forecast cases for the next day). 
I would like to describe 4 important points for this kind of modeling:
- cross-validation strategy
- target
- features
- models

#### Target

This is an important step to decide. However, we can experiment with different approaches here. I see following sane approaches:
- value of Confirmed at step (t+1)
- diff value of Confirmed at step (t+1): Confirmed(t+1) - Confirmed(t)
- MA of Confirmed on some window at step (t+1)

#### Cross-validation

This is also an important step, as this should be a simulation of how model would perform in production. We can't use simple KFold for time series, better to use something more complicated - like roll-forward cross validation. I would like to try Time Series Split Cross-Validation or Blocked Cross-Validation.


#### Features

Features I see for this task are:
- lag matrices of target (Confirmed)
- diff matrices of target (Confirmed)
- Moving averages (MA, EMA, etc.)
- Features available for dataset: Deaths, Recovered, Active, Incident_Rate, Case_Fatality_Ratio with their aggregation on some window(s) and cumulative ratios where its sane to do so
- Externally added features: general country info - wealth, GDP, total population, population density (use if fitting one-model-fits-all approach)
- Handcrafted features like exp, pow(s) of a target, local predictions with simple models (like lin regression prediction from past week data) etc.
- arrgegated features from neighboring countries (AVG number of cases, deaths, etc)

No matter how much features we generate and use, we always have an ability to select best ones using:
- feature importances (e.g. from RF model)
- recursive feature elimination (or any other feature selection method of your choice)

Important note here to consider: during feature generation we should be very careful to not impute future data to feature matrix (data leakege), as it will result into surprisingly great results during development but will fail to generalise at all.

#### Models

This is the most fascinating step, as it includes a lot of experimentation.

I would like to stick to some classical ML algorithms (tree-based: RF, Gradient boosting; linear: regression, SVM) and also try to fit some DL models that fit ok to TS tasks (simple MLP, multi-head MLP to cover multi-task learning features; RNNs: vanilla RNN, LSTM, GRU; 1D-CNN; Attention-based transformers)

But, again, following Occam's razor and Python principles: simple is better then complicated

Training pipeline is simple: using described CV strategy: fit a model on traing split, evaluate it on validation split. Repeat for all splits. Record model results to some external file for further analysis and model selection.

At the end, with best model(s) selected, we can tune hyperparamethers with some search algorithm (random search, grid search, Bayesian optimisation, genetic algorithm - I prefer using ray.tune or pygad for this)

During evaluation it's important to track that model doesn't just repeat past value (lazy prediction - output Confirmed(t) as a prediction) and does not just learns mean value of series (again lazy model can just learn mean(Confirmed(0:t))).